In [1]:
%matplotlib inline

In [2]:
from typing import List, Union

from qiskit import pulse
from qiskit.pulse.reschedule import pad
import qiskit.pulse.pulse_lib as pulse_lib
from qiskit.pulse import Schedule, DriveChannel, MeasureChannel, FrameChange, Delay

import numpy as np

In [3]:
sched = Schedule()
f = pulse_lib.gaussian(100, 1.0, 10.0)
sched += f(DriveChannel(0))
sched += FrameChange(np.pi/2)(DriveChannel(0))
sched |= f(DriveChannel(0)).shift(100)
sched.draw()

TypeError: unsupported operand type(s) for +=: 'NoneType' and 'FrameChangeInstruction'

In [ ]:
ramsey = pulse.Schedule()
x90 = pulse_lib.gaussian_square(100, 0.5j, 50, 20)
ramsey += x90(DriveChannel(0))
ramsey += Delay(100)(DriveChannel(0))
ramsey += x90(DriveChannel(0))
ramsey += x90(DriveChannel(1)) << 200
align = [f(DriveChannel(0)), f(DriveChannel(0)), ramsey, f(DriveChannel(1))]

# Align left

In [ ]:
def push_append(this: List[pulse.ScheduleComponent], 
                other: List[pulse.ScheduleComponent], 
                mutate=True) -> pulse.Schedule:
        r"""Return a new schedule with `schedule` inserted at the maximum time over
        all channels shared between `self` and `schedule`.

       $t = \textrm{max}({x.stop\_time |x \in self.channels \cap schedule.channels})$

        Args:
            schedule: schedule to be appended
            buffer: Whether to obey buffer when appending
            mutate: Instead of returning a new schedule the call will be performed 
                by mutating `this`.
        """
        common_channels = set(this.channels) & set(other.channels)
        
        channels = {}
        min_ch_slack = np.inf
        max_common_time = this.ch_stop_time(*common_channels)
        ch_slacks = [this.ch_stop_time(channel)+other.ch_start_time(channel) 
                     for channel in common_channels]
           
        insert_time = this.start_time + min(ch_slacks, default=0)
        
        return this.insert(insert_time, other, mutate=mutate)
    
def align_left(*instructions: List[Union[pulse.Instruction, pulse.Schedule]], 
               mutate=True) -> pulse.Schedule:
    """Align a list of pulse instructions on the left
    
    Args:
        instructions: List of pulse instructions to align.
        mutate: Instead of returning a new schedule the call will be performed 
                by mutating `this`.
    
    Returns:
        pulse.Schedule
    """
    aligned = pulse.Schedule()
    for instruction in instructions:
        aligned = push_append(aligned, instruction, mutate=mutate)
    
    return aligned

def align_left_padded(*instructions: List[pulse.ScheduleComponent],
                      mutate=True) -> pulse.Schedule:
    """Align a list of pulse instructions on the left with padding
    
    Args:
        instructions: List of pulse instructions to align.
        mutate: Instead of returning a new schedule the call will be performed 
                by mutating `this`.
    
    Returns:
        pulse.Schedule
    """
    aligned = align_left(*instructions, mutate=mutate)
    return pad(aligned, mutate=mutate)

def align_right(*instructions: List[pulse.ScheduleComponent],
                mutate=True) -> pulse.Schedule:
    """Align a list of pulse instructions on the right
    
    Args:
        instructions: List of pulse instructions to align.
        mutate: Instead of returning a new schedule the call will be performed 
                by mutating `this`.
    
    Returns:
        pulse.Schedule
    """
    left_aligned = align_left(*instructions, mutate=False)
    max_duration = 0
    
    channel_durations = {}
    for channel in left_aligned.channels:
        channel_sched = left_aligned.filter(channels=[channel])
        channel_duration = channel_sched.duration
        channel_durations[channel] = channel_sched.duration
        max_duration = max(max_duration, channel_duration)
    
    aligned = pulse.Schedule()
    for instr_time, instruction in left_aligned.instructions:
        instr_max_dur = max(channel_durations[channel] for channel in instruction.channels)
        instr_delayed_time = max_duration - instr_max_dur + instr_time
        aligned.insert(instr_delayed_time, instruction, mutate=mutate)
        
    return aligned

def align_right_padded(*instructions: List[pulse.ScheduleComponent],
                       mutate=True) -> pulse.Schedule:
    """Align a list of pulse instructions on the right with padding
    
    Args:
        instructions: List of pulse instructions to align.
        mutate: Instead of returning a new schedule the call will be performed 
                by mutating `this`.
    
    Returns:
        pulse.Schedule
    """
    aligned = align_right(*instructions, mutate=mutate)
    return pad(aligned, mutate=mutate)

def align_in_sequence(*instructions: List[pulse.ScheduleComponent],
                      mutate=True) -> pulse.Schedule:
    """Align a list of pulse instructions sequentially in time.
    Args:
        instructions: List of pulse instructions to align.
        mutate: Instead of returning a new schedule the call will be performed 
                by mutating `this`.
    Returns:
        A new pulse schedule with instructions`
    """
    aligned = pulse.Schedule()
    for instruction in instructions:
        if mutate:
            aligned.insert(aligned.duration, instruction, mutate=mutate)
    return aligned

In [ ]:
aligned_left._children

In [ ]:
aligned_left = align_left_padded(f(DriveChannel(1)), *align)
aligned_left.draw()

In [ ]:
aligned_left = align_left(f(DriveChannel(1)), ramsey)
aligned_left.draw()

In [ ]:
aligned_right = align_right(*align)
aligned_right.draw()

In [ ]:
aligned_left_padded = align_left_padded(*align)
aligned_left_padded.draw()
added =  aligned_left_padded + f(DriveChannel(1))
added.draw()

In [ ]:
added =  aligned_left + f(DriveChannel(1))
added.draw()

In [ ]:
aligned_right_padded = align_right_padded(*align)
aligned_right_padded.draw()
added =  f(DriveChannel(1)) + aligned_right_padded 
added.draw()

In [ ]:
added =  align_left(f(DriveChannel(1)), aligned_right)
added.draw()

In [ ]:
align_in_sequence(*align).draw()